In [2]:
import matplotlib.pyplot as plt
import torch
from   torch import nn
from   torch.utils.data import DataLoader
from   torchvision import datasets
from   torchvision.transforms import Compose, Lambda, ToTensor

In [3]:
!ls .

quickstart.py.ipynb


In [5]:
training_data = datasets.FashionMNIST(
    root='data', train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(
    root='data', train=False, download=True, transform=ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
!ls .

data                quickstart.py.ipynb


In [8]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print('Shape X: [N, C, H, W]:', X.shape)
    print('Shape y:', y.shape, y.dtype)
    break

Shape X: [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape y: torch.Size([64]) torch.int64


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28 * 28, 512),
                                               nn.ReLU(),
                                               nn.Linear(512, 512),
                                               nn.ReLU(),
                                               nn.Linear(512, 10),
                                               nn.ReLU())
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [13]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [19]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        preds = model(X)
        loss = loss_fn(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [22]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    n_batches = len(dataloader)
    model.eval()
    test_loss, correct, = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            preds = model(X)
            test_loss += loss_fn(preds, y).item()
            correct += (preds.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= n_batches
    correct /= size
    print(f'Test Err:\n Accuracy: {(100 * correct):>0.1f}%\n'
          f' Avg loss: {test_loss:>8f}')

In [23]:
EPOCHS = 5
for t in range(EPOCHS):
    print(f'Epoch {t + 1}\n{"-" * 30}')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print('Done')

Epoch 1
------------------------------
loss: 2.177487 [    0/60000]
loss: 2.163764 [ 6400/60000]
loss: 2.115241 [12800/60000]
loss: 2.131296 [19200/60000]
loss: 2.066601 [25600/60000]
loss: 2.029877 [32000/60000]
loss: 2.055859 [38400/60000]
loss: 1.977515 [44800/60000]
loss: 1.994249 [51200/60000]
loss: 1.963384 [57600/60000]
Test Err:
 Accuracy: 59.4%
 Avg loss: 1.943860
Epoch 2
------------------------------
loss: 1.978030 [    0/60000]
loss: 1.947600 [ 6400/60000]
loss: 1.850052 [12800/60000]
loss: 1.890809 [19200/60000]
loss: 1.778604 [25600/60000]
loss: 1.729997 [32000/60000]
loss: 1.778346 [38400/60000]
loss: 1.660954 [44800/60000]
loss: 1.708575 [51200/60000]
loss: 1.666088 [57600/60000]
Test Err:
 Accuracy: 61.5%
 Avg loss: 1.654303
Epoch 3
------------------------------
loss: 1.732108 [    0/60000]
loss: 1.692995 [ 6400/60000]
loss: 1.566669 [12800/60000]
loss: 1.619206 [19200/60000]
loss: 1.498387 [25600/60000]
loss: 1.458142 [32000/60000]
loss: 1.512231 [38400/60000]
loss: 

In [24]:
torch.save(model.state_dict(), 'model.pth')

In [25]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [26]:
classes = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 
    'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [27]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: {predicted}; Actual: {actual}')

Predicted: Ankle boot; Actual: Ankle boot
